In [371]:
import pandas as pd
from pandas import ExcelWriter
from pandas import ExcelFile
import numpy as np
from splinter import Browser
from io import StringIO
import time
import requests
import lxml
import datetime as dt
import openpyxl
from openpyxl import workboo
from openpyxl import load_workbook

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common import keys
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

import warnings
warnings.filterwarnings("ignore")

In [24]:
#install new Chrome webdriver
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))



====== WebDriver manager ======
Current google-chrome version is 101.0.4951
Get LATEST chromedriver version for 101.0.4951 google-chrome
Driver [C:\Users\Will_Boisseau\.wdm\drivers\chromedriver\win32\101.0.4951.41\chromedriver.exe] found in cache


In [25]:
# Scrape Morningstar

# Create webdriver
driver=webdriver.Chrome()
driver.get("https://www.morningstar.com/etfs/xnas/acwi/portfolio")
#print(value)

# Close the browser after scraping
#driver.quit()


In [28]:
time.sleep(2)

In [33]:
rows = 1 +len(driver.find_elements_by_xpath('//*[@id="__layout"]/div/div[2]/div[3]/div/main/div[2]/div/div/div[1]/div[2]/sal-components/section/div/div/div[1]/sal-components-pillar-cards-process/div/div[2]/div/div[6]/div[1]/div/sal-components-mip-exposure/div/div/div/div[2]/div[1]/sal-components-mip-sector-exposure/div/div/div/div[1]/div/div[2]/div[1]/table/tbody/tr'))
cols=len(driver.find_elements_by_xpath('//*[@id="__layout"]/div/div[2]/div[3]/div/main/div[2]/div/div/div[1]/div[2]/sal-components/section/div/div/div[1]/sal-components-pillar-cards-process/div/div[2]/div/div[6]/div[1]/div/sal-components-mip-exposure/div/div/div/div[2]/div[1]/sal-components-mip-sector-exposure/div/div/div/div[1]/div/div[2]/div[1]/table/tbody/tr[1]/td'))
list_1 = []
for i in range(1, rows):
    list_2 = []
    for j in range(1, cols+1):
        value = driver.find_element_by_xpath('//*[@id="__layout"]/div/div[2]/div[3]/div/main/div[2]/div/div/div[1]/div[2]/sal-components/section/div/div/div[1]/sal-components-pillar-cards-process/div/div[2]/div/div[6]/div[1]/div/sal-components-mip-exposure/div/div/div/div[2]/div[1]/sal-components-mip-sector-exposure/div/div/div/div[1]/div/div[2]/div[1]/table/tbody/tr['+str(i)+']/td['+str(j)+']').text
        print(value, end='     ')
        list_2.append(value)
    list_1.append(list_2)
    print()

Basic Materials     4.85     5.40     
Consumer Cyclical     10.53     10.46     
Financial Services     15.61     14.07     
Real Estate     2.86     2.42     
Communication Services     8.17     7.12     
Energy     5.02     4.21     
Industrials     9.46     14.27     
Technology     20.05     18.00     
Consumer Defensive     7.87     7.75     
Healthcare     12.53     12.37     
Utilities     3.03     3.94     


In [55]:
#put the list into a dataframe
df=pd.DataFrame(list_1)
df

,0,1,2
0,Basic Materials,4.85,5.40
1,Consumer Cyclical,10.53,10.46
2,Financial Services,15.61,14.07
3,Real Estate,2.86,2.42
4,Communication Services,8.17,7.12
5,Energy,5.02,4.21
6,Industrials,9.46,14.27
7,Technology,20.05,18.00
8,Consumer Defensive,7.87,7.75
9,Healthcare,12.53,12.37


In [246]:
#drop the last column, rename remaining columns

df2= df.drop([2],axis=1)
df2.rename(columns={0:'GICS Sector',1:'ACWI Weight'},inplace=True)

#rename sectors to match GICS sectors
df2['GICS Sector']=df2['GICS Sector'].replace({'Consumer Cyclical':'Consumer Discretionary','Basic Materials':'Materials','Financial Services':'Financials','Technology':'Information Technology','Consumer Defensive':'Consumer Staples'})
df2



,GICS Sector,ACWI Weight
0,Materials,4.85
1,Consumer Discretionary,10.53
2,Financials,15.61
3,Real Estate,2.86
4,Communication Services,8.17
5,Energy,5.02
6,Industrials,9.46
7,Information Technology,20.05
8,Consumer Staples,7.87
9,Healthcare,12.53


In [332]:
#get latest weightings for each holding

today = dt.datetime.today().strftime("%m.%d.%Y")
ClosePath=f'C:/Users/Will_Boisseau/OneDrive - Georgetown University/Graduate Investment Fund (GIF)/All Things Investing/Performance & Research/Performance/Brokerage Data_for all club members/GIF_historical_closes05.25.2022.xlsx'

weightdf=pd.read_excel(ClosePath,sheet_name='GIF_YTD_Perf',index_col=0)

#count number of columns. Today's market values are at index cols-6
cols=weightdf.shape[1]
cols

#dataframe should just be ticker and current value
weightdf=weightdf.iloc[:,[0,cols-6]]

#calculate weight of each position based on total portfolio MV
MVsum=weightdf['Latest_Total_Value'].sum()
weightdf['GIF_Weight']=weightdf['Latest_Total_Value']/MVsum
weightdf.tail()


,Ticker,Latest_Total_Value,GIF_Weight
28,RYE,32939.359009,0.043440
29,RYF,27413.920227,0.036153
30,VOX,11502.299614,0.015169
31,VRTX,37347.800903,0.049254
32,Cash_Balance,32463.840000,0.042813


In [362]:
#get path of cost basis file, which has GICS sectors listed
BasisPath=f'C:/Users/Will_Boisseau/OneDrive - Georgetown University/Graduate Investment Fund (GIF)/All Things Investing/Performance & Research/Performance/Brokerage Data_for all club members/GIF_CostBasis_asof{today}.xlsx'

#get GICS sector from above file. Then dataframe should just be ticker and GICS sector
basisdf=pd.read_excel(BasisPath,sheet_name='GIF_attributes-cost',index_col=0)
basisdf=basisdf.iloc[:,[1,13]]

#combine the frames and match ticker w/ correct sector
combdf=pd.merge(weightdf,basisdf,on="Ticker",how='left')
combdf[['GICS_Sector']]=combdf[['GICS_Sector']].fillna("Cash")
combdf.rename(columns={'GICS_Sector':'GICS Sector','GIF_Weight':'GIF Weight'},inplace=True)

#remove excess whitespace in the GICS Sector fields
combdf['GICS Sector'] = combdf['GICS Sector'].str.strip()
combdf


,Ticker,Latest_Total_Value,GIF Weight,GICS Sector
0,AON,34225.228760,0.045136,Financials
1,AAPL,34500.901794,0.045499,Information Technology
2,BRK-B,24531.201172,0.032351,Financials
3,CZR,15823.500252,0.020868,Consumer Discretionary
4,CAT,18922.540222,0.024955,Industrials
5,CI,46877.741333,0.061822,Healthcare
6,DHI,14784.749794,0.019498,Consumer Discretionary
7,HON,12407.679688,0.016363,Industrials
8,RHS,13142.399902,0.017332,Consumer Staples
9,MU,22878.100983,0.030171,Information Technology


In [363]:
#calculate weight of each sector, drop the total value column

GIFsecdf=combdf.groupby('GICS Sector').sum()
GIFsecdf= GIFsecdf.drop(["Latest_Total_Value"],axis=1)
GIFsecdf


,GIF Weight
GICS Sector,
Cash,0.042813
Communication Services,0.047300
Consumer Discretionary,0.092615
Consumer Staples,0.067707
Energy,0.043440
Financials,0.160846
Healthcare,0.154878
Industrials,0.090567
Information Technology,0.168462


In [382]:
#combine the GIF and ACWI weight dataframes

#combine the frames and match ticker w/ correct sector, use 0 for ACWI Cash weight, cast both weights as float,
#state weights as %, compute active weight as GIF-ACWI

activedf=pd.merge(GIFsecdf,df2,on="GICS Sector",how='left')

activedf['ACWI Weight'] = activedf['ACWI Weight'].fillna(0)
activedf['ACWI Weight'] = activedf['ACWI Weight'].astype(float)
activedf['GIF Weight']=activedf['GIF Weight']*100
activedf['GIF Weight'] = activedf['GIF Weight'].astype(float)
activedf['Active Weight']=activedf['GIF Weight']-activedf['ACWI Weight']
activedf



,GICS Sector,GIF Weight,ACWI Weight,Active Weight
0,Cash,4.281295,0.00,4.281295
1,Communication Services,4.729962,8.17,-3.440038
2,Consumer Discretionary,9.261525,10.53,-1.268475
3,Consumer Staples,6.770736,7.87,-1.099264
4,Energy,4.344006,5.02,-0.675994
5,Financials,16.084643,15.61,0.474643
6,Healthcare,15.487771,12.53,2.957771
7,Industrials,9.056746,9.46,-0.403254
8,Information Technology,16.846176,20.05,-3.203824
9,Materials,4.995942,4.85,0.145942


In [396]:
# Create a Pandas Excel writer using XlsxWriter as the engine.

    
writer = pd.ExcelWriter('C:/Users/Will_Boisseau/OneDrive - Georgetown University/Graduate Investment Fund (GIF)/All Things Investing/Performance & Research/Performance/Brokerage Data_for all club members/Active Weights/Active_Weights_asof05.25.2022.xlsx', engine='xlsxwriter')
activedf.to_excel(writer,sheet_name='Active Weights')
writer.save()
writer


#GIF_combined_cost['Report_data'] = dt.datetime.today().strftime("%m/%d/%Y")
#today = dt.datetime.today().strftime("%m.%d.%Y")
#writer = pd.ExcelWriter(f"Output_Data/GIF_CostBasis_asof{today}.xlsx", engine='xlsxwriter')


# Write each dataframe to a different worksheet.
#GIF_combined_attrib.to_excel(writer, sheet_name='GIF_attributes-cost')
#GIF_combined_cost.to_excel(writer, sheet_name='cost_raw')

# Close the Pandas Excel writer and output the Excel file.
#writer.save()
